In [115]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time

In [116]:
SCROLL_PAUSE_TIME = 0.5

In [117]:
def collect_sentences(original_post, responses):
    print(original_post)
    print()
    print(responses)

In [118]:
# NOTE: may need to remove \n characters depending on semantics of how they parsed sentences. Also
# may need to replace the unicode vestigial characters with empty string instead of space.

def scrape_url(conversation_url):
    print("Conversation URL: {}".format(conversation_url))
        
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("headless")

    driver = webdriver.Chrome(options = chrome_options)
    driver.get(conversation_url + "?sort_by=oldest")

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source)
    
    original_post = soup.find("div", {"id" : "subject_msg"}).text
    html_id = "post_show_" + conversation_url.split("/")[-1]
    current_conversation = soup.find("div", {"id" : html_id})
    
    responses = [str(response.text) for response in current_conversation.find_all("div", {"class" : "resp_body"})]
    responses = [response.replace(u"\xa0", u" ").strip() for response in responses]
    
    collect_sentences(original_post, responses) 

In [119]:
def scrape_conversations(disease):
    conversations = pd.read_csv("gold_standard/conversations/" + 
                                disease + ".tsv", delimiter = "\t")
    
    for url in conversations["url"]:
        scrape_url(url)
        break

In [120]:
def driver():
    diseases = pd.read_csv("gold_standard/diseases/diseases.tsv", delimiter = "\t")
    
    for disease in diseases["disease_id"]:
        print("Disease ID: {}".format(disease))
        scrape_conversations(disease)
        break

In [121]:
driver()

Disease ID: HIF2016_DIS00
Conversation URL: http://www.medhelp.org/posts/Allergies---Food/Does-anyone-have-an-allergy-to-coffee/show/1550820

          I have been a coffee drinker on and off since I was little.  I notice now that I am older, and perhaps I have always had this issue just did not correlate the connection between the two, but I have to wonder if it is possible for me to have an allergy to coffee.

When I drink it, if I drink about 2 cups a day spaced out, it takes about a week for me to get these symptoms:

- sore throat
- increased nasal drainage and nasal drip in the back of my throat
- sometimes slight eye swelling
- slight cough

I always kind of said oh it's just allergies from pollen or I am catching a cold... but in fact when I do not consume coffee, I am back to normal in about 3 days.  I tried to find some information on this but it seems hard to read up on.

Would be great if anyone had any feedback on this.  Thanks so much.
      

['Are you drinking decaf or 